<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/multi-discriminator%20GAN%200830.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Discriminator GAN


ABSTRACT

----


#4. Implementation


## 4.1 Í∏∞Î≥∏ ÏÑ§Ï†ï...

In [1]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!pip install keybert
!pip3 install transformers
!pip3 install sentence-transformers

#!pip install sentence-transformers

     |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 2.6 MB 5.1 MB/s 
     |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 895 kB 48.3 MB/s 
     |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 3.3 MB 35.7 MB/s 
     |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 636 kB 39.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 85 kB 2.5 MB/s 
     |‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 1.2 MB 15.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=271cc2

In [3]:
import matplotlib.pyplot as plt
# set seeds for reproducability
from numpy.random import seed
#seed(1)

import pandas as pd
import numpy as np
import string, os 

import urllib.request
import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
# Print iterations progress
class ProgressBar:

    def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '|', printEnd = "\r"):
        self.total = total
        self.prefix = prefix
        self.suffix = suffix
        self.decimals = decimals
        self.length = length
        self.fill = fill
        self.printEnd = printEnd
        self.ite = 0
        self.back_filledLength = 0

    def printProgress(self,iteration, text):
        self.ite += iteration
        percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))
        filledLength = int(self.length * self.ite // self.total)
        bar = self.fill * filledLength + '.' * (self.length - filledLength)
        if filledLength > self.back_filledLength or percent == 100:
            print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
            # Print New Line on Complete
            if self.ite == self.total: 
                print()
        self.back_filledLength = filledLength    

In [6]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

##4.2 Grammar Discriminator Class

In [7]:
from transformers import BertTokenizer, BertTokenizerFast,AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split

import time
import random
import datetime

# Í∞ÑÎã®Ìïú Ï†ÑÏ≤òÎ¶¨
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    txt = txt.replace(' .','.')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def shuffling(txt):
    txt_list = txt.split(' ')
    random.shuffle(txt_list)
    return ' '.join(txt_list)

def collect_training_dataset_for_grammar_discriminator(sentences_dataset):

    sentences = []
    labels = []

    for txtss in sentences_dataset:
        txtss = clean_text(txtss)
        txts = txtss.strip().split('.')
        for txt in txts:  
            txt = txt.strip()
            if len(txt) > 10:
                #ko_grammar_dataset.append([txt,1])
                txt = txt.replace('.','')
                tf = random.choice([True,False])
                # Ï†ïÏÉÅ ÎòêÎäî ÎπÑÏ†ïÏÉÅ ÎëòÏ§ëÏóê ÌïòÎÇòÎßå Îç∞Ïù¥ÌÑ∞ÏÖãÏóê Ï∂îÍ∞Ä
                if (tf):
                    sentences.append(txt) # '.'Ïùò ÏúÑÏπòÎ•º Î≥¥Í≥† True, FalseÎ•º ÌåêÎã® ÌïòÍ∏∞ ÎïúÏóê...
                    labels.append(1)
                else:
                    sentences.append(shuffling(txt))
                    labels.append(0)

    return sentences,labels

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

class Grammar_Discriminator:


    def __init__(self, pretraoned_kobert_model_name='kykim/bert-kor-base', input_dir=None):

        if input_dir is None:
            self.tokenizer = BertTokenizerFast.from_pretrained(pretraoned_kobert_model_name)
            self.discriminator = BertForSequenceClassification.from_pretrained(
                                    pretraoned_kobert_model_name, # Use the 12-layer BERT model, with an uncased vocab.
                                    num_labels = 2, # The number of output labels--2 for binary classification.
                                                    # You can increase this for multi-class tasks.   
                                    output_attentions = False, # Whether the model returns attentions weights.
                                    output_hidden_states = False, # Whether the model returns all hidden-states.
                                )            
        else:
            self.__load_model(input_dir)




    def set_dataset(self, sentences,labels):
        # Print the original sentence.
        print(' Original: ', sentences[0])

        # Tokenize all of the sentences and map the tokens to thier word IDs.
        input_ids = []
        attention_masks = []

        # For every sentence...
        for sent in sentences:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = self.tokenizer.encode_plus(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 64,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                                truncation = True,
                        )
            
            # Add the encoded sentence to the list.    
            input_ids.append(encoded_dict['input_ids'])
            
            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])

        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        labels = torch.tensor(labels)

        # Print sentence 0, now as a list of IDs.
        print('Original: ', sentences[0])
        print('Token IDs:', input_ids[0])

        # Training & Validation Split
        # Divide up our training set to use 90% for training and 10% for validation.

        # Combine the training inputs into a TensorDataset.
        dataset = TensorDataset(input_ids, attention_masks, labels)

        # Create a 90-10 train-validation split.

        # Calculate the number of samples to include in each set.
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size

        # Divide the dataset by randomly selecting samples.
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        print('{:>5,} training samples'.format(train_size))
        print('{:>5,} validation samples'.format(val_size))

        # The DataLoader needs to know our batch size for training, so we specify it 
        # here. For fine-tuning BERT on a specific task, the authors recommend a batch 
        # size of 16 or 32.
        self.batch_size = 32

        # Create the DataLoaders for our training and validation sets.
        # We'll take training samples in random order. 
        self.train_dataloader = DataLoader(
                    train_dataset,  # The training samples.
                    sampler = RandomSampler(train_dataset), # Select batches randomly
                    batch_size = self.batch_size # Trains with this batch size.
                )

        # For validation the order doesn't matter, so we'll just read them sequentially.
        self.validation_dataloader = DataLoader(
                    val_dataset, # The validation samples.
                    sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                    batch_size = self.batch_size # Evaluate with this batch size.
                )        



    def train(self,epochs=4):
        # Tell pytorch to run this model on the GPU.
        self.discriminator.cuda()

        # Get all of the model's parameters as a list of tuples.
        params = list(self.discriminator.named_parameters())

        print('The BERT model has {:} different named parameters.\n'.format(len(params)))

        print('==== Embedding Layer ====\n')

        for p in params[0:5]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

        print('\n==== First Transformer ====\n')

        for p in params[5:21]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

        print('\n==== Output Layer ====\n')

        for p in params[-4:]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))  

        # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
        # I believe the 'W' stands for 'Weight Decay fix"
        self.optimizer = AdamW(self.discriminator.parameters(),
                        lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                        eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                        )

        # Number of training epochs. The BERT authors recommend between 2 and 4. 
        # We chose to run for 4, but we'll see later that this may be over-fitting the
        # training data.
        #epochs = 2

        # Total number of training steps is [number of batches] x [number of epochs]. 
        # (Note that this is not the same as the number of training samples).
        total_steps = len(self.train_dataloader) * epochs

        # Create the learning rate scheduler.
        scheduler = get_linear_schedule_with_warmup(self.optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)
            
        # This training code is based on the `run_glue.py` script here:
        # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

        # Set the seed value all over the place to make this reproducible.
        seed_val = 42

        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)

        # We'll store a number of quantities such as training and validation loss, 
        # validation accuracy, and timings.
        training_stats = []

        # Measure the total training time for the whole run.
        total_t0 = time.time()

        # For each epoch...
        for epoch_i in range(0, epochs):
            
            # ========================================
            #               Training
            # ========================================
            
            # Perform one full pass over the training set.

            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            print('Training...')

            # Measure how long the training epoch takes.
            t0 = time.time()

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode. Don't be mislead--the call to 
            # `train` just changes the *mode*, it doesn't *perform* the training.
            # `dropout` and `batchnorm` layers behave differently during training
            # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
            self.discriminator.train()

            # For each batch of training data...
            for step, batch in enumerate(self.train_dataloader):

                # Progress update every 40 batches.
                if step % 40 == 0 and not step == 0:
                    # Calculate elapsed time in minutes.
                    elapsed = format_time(time.time() - t0)
                    
                    # Report progress.
                    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(self.train_dataloader), elapsed))

                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using the 
                # `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                # Always clear any previously calculated gradients before performing a
                # backward pass. PyTorch doesn't do this automatically because 
                # accumulating the gradients is "convenient while training RNNs". 
                # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
                self.discriminator.zero_grad()        

                # Perform a forward pass (evaluate the model on this training batch).
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # It returns different numbers of parameters depending on what arguments
                # arge given and what flags are set. For our useage here, it returns
                # the loss (because we provided labels) and the "logits"--the model
                # outputs prior to activation.
                outputs = self.discriminator(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask, 
                                    labels=b_labels)
                loss, logits = outputs.loss, outputs.logits
                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end. `loss` is a Tensor containing a
                # single value; the `.item()` function just returns the Python value 
                # from the tensor.
                total_train_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Clip the norm of the gradients to 1.0.
                # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(self.discriminator.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                self.optimizer.step()

                # Update the learning rate.
                scheduler.step()

            # Calculate the average loss over all of the batches.
            avg_train_loss = total_train_loss / len(self.train_dataloader)            
            
            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)

            print("")
            print("  Average training loss: {0:.2f}".format(avg_train_loss))
            print("  Training epcoh took: {:}".format(training_time))
                
            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure our performance on
            # our validation set.

            print("")
            print("Running Validation...")

            t0 = time.time()

            # Put the model in evaluation mode--the dropout layers behave differently
            # during evaluation.
            self.discriminator.eval()

            # Tracking variables 
            total_eval_accuracy = 0
            total_eval_loss = 0
            nb_eval_steps = 0

            # Evaluate data for one epoch
            for batch in self.validation_dataloader:
                
                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using 
                # the `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)
                
                # Tell pytorch not to bother with constructing the compute graph during
                # the forward pass, since this is only needed for backprop (training).
                with torch.no_grad():        

                    # Forward pass, calculate logit predictions.
                    # token_type_ids is the same as the "segment ids", which 
                    # differentiates sentence 1 and 2 in 2-sentence tasks.
                    # The documentation for this `model` function is here: 
                    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                    # Get the "logits" output by the model. The "logits" are the output
                    # values prior to applying an activation function like the softmax.
                    outputs = self.discriminator(b_input_ids, 
                                        token_type_ids=None, 
                                        attention_mask=b_input_mask,
                                        labels=b_labels)
                loss, logits = outputs.loss, outputs.logits
                # Accumulate the validation loss.
                total_eval_loss += loss.item()

                # Move logits and labels to CPU
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()

                # Calculate the accuracy for this batch of test sentences, and
                # accumulate it over all batches.
                total_eval_accuracy += flat_accuracy(logits, label_ids)
                

            # Report the final accuracy for this validation run.
            avg_val_accuracy = total_eval_accuracy / len(self.validation_dataloader)
            print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

            # Calculate the average loss over all of the batches.
            avg_val_loss = total_eval_loss / len(self.validation_dataloader)
            
            # Measure how long the validation run took.
            validation_time = format_time(time.time() - t0)
            
            print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            print("  Validation took: {:}".format(validation_time))

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'epoch': epoch_i + 1,
                    'Training Loss': avg_train_loss,
                    'Valid. Loss': avg_val_loss,
                    'Valid. Accur.': avg_val_accuracy,
                    'Training Time': training_time,
                    'Validation Time': validation_time
                }
            )

        print("")
        print("Training complete!")

        print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
            

        return training_stats

    def save_model(self, output_dir = './model_save/'):
        # Create output directory if needed
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print("Saving model to %s" % output_dir)

        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = self.discriminator.module if hasattr(self.discriminator, 'module') else self.discriminator  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        self.tokenizer.save_pretrained(output_dir)

        # Good practice: save your training arguments together with the trained model
        # torch.save(args, os.path.join(output_dir, 'training_args.bin'))

    def __load_model(self, input_dir = './drive/MyDrive/Colab Notebooks/summary/en_grammar_check_model'):
        print('Loading BERT tokenizer...')
        self.tokenizer = BertTokenizerFast.from_pretrained(input_dir)
        self.discriminator = BertForSequenceClassification.from_pretrained(input_dir)

    def transfer_learning(self, sentences, train_for = True):
        
        input_ids = []
        attention_masks = []

        # For every sentence...
        for sent in sentences:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = self.tokenizer.encode_plus(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 64,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                                truncation = True,
                        )
            # Add the encoded sentence to the list.    
            input_ids.append(encoded_dict['input_ids'])

            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])
        
        if train_for:
            b_labels = torch.ones(len(sentences),dtype=torch.long).to(device)
        else:
            b_labels = torch.zeros(len(sentences),dtype=torch.long).to(device)
        #print(b_labels)
        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids, dim=0).to(device)
        attention_masks = torch.cat(attention_masks, dim=0).to(device)    
        #if str(discriminator1.device) == 'cpu':
        #    pass
        #else:
        #    input_ids = input_ids.to(device)
        #    attention_masks = attention_masks.to(device)        

        outputs = self.discriminator(input_ids, 
                            token_type_ids=None, 
                            attention_mask=attention_masks, 
                                labels=b_labels)

        #print(outputs)
        #return torch.sigmoid(outputs[0][:,1])
        #return outputs[0][:,1]
        return outputs['loss'], outputs['logits']


# Î¨∏Î≤ï ÌïôÏäµÏùÑ ÏúÑÌïú Îç∞Ïù¥ÌÑ∞ÏÖã Íµ¨ÏÑ±

In [8]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
df

title                                           contents
0                  Î¨∏ ÎåÄÌÜµÎ†π Î≥ÄÏ∞ΩÌù† Íµ≠ÌÜ†Ïû•Í¥Ä ÏÇ¨ÏùòÌëúÎ™Ö ÏÇ¨Ïã§ÏÉÅ ÏàòÏö©  Ï†ïÎßåÌò∏ Íµ≠ÎØºÏÜåÌÜµÏàòÏÑùÏù¥ 12Ïùº Ïò§ÌõÑ Ï≤≠ÏôÄÎåÄ Ï∂òÏ∂îÍ¥Ä ÎåÄÎ∏åÎ¶¨ÌïëÎ£∏ÏóêÏÑú Î≥ÄÏ∞ΩÌù† Íµ≠ÌÜ†Î∂Ä Ïû•Í¥Ä ...
1                              Í≥ÑÍ∏âÏû• ÏàòÏó¨ÌïòÎäî Î¨∏ ÎåÄÌÜµÎ†π  (ÏïÑÏÇ∞Îâ¥Ïä§1) Ïù¥Í¥ëÌò∏ Í∏∞Ïûê Î¨∏Ïû¨Ïù∏ ÎåÄÌÜµÎ†πÏù¥ 12Ïùº Ïò§ÌõÑ Ï∂©ÎÇ® ÏïÑÏÇ∞Ïãú Í≤ΩÏ∞∞ÎåÄÌïôÏóêÏÑú Ïó¥...
2                              Í≥ÑÍ∏âÏû• ÏàòÏó¨ÌïòÎäî Î¨∏ ÎåÄÌÜµÎ†π  (ÏïÑÏÇ∞Îâ¥Ïä§1) Ïù¥Í¥ëÌò∏ Í∏∞Ïûê Î¨∏Ïû¨Ïù∏ ÎåÄÌÜµÎ†πÏù¥ 12Ïùº Ïò§ÌõÑ Ï∂©ÎÇ® ÏïÑÏÇ∞Ïãú Í≤ΩÏ∞∞ÎåÄÌïôÏóêÏÑú Ïó¥...
3                           ÏàòÏÉÅÏûê Î©îÎã¨ Í±∏Ïñ¥Ï£ºÎäî Î¨∏ ÎåÄÌÜµÎ†π  (ÏïÑÏÇ∞Îâ¥Ïä§1) Ïù¥Í¥ëÌò∏ Í∏∞Ïûê Î¨∏Ïû¨Ïù∏ ÎåÄÌÜµÎ†πÏù¥ 12Ïùº Ïò§ÌõÑ Ï∂©ÎÇ® ÏïÑÏÇ∞Ïãú Í≤ΩÏ∞∞ÎåÄÌïôÏóêÏÑú Ïó¥...
4                          Ï†ïÎ™ΩÍµ¨ ÏÑúÏö∏ÏïÑÏÇ∞Î≥ëÏõêÏóê 50Ïñµ ÏæåÏ≤ô  Ïù∏Ïû¨ ÏñëÏÑ±¬∑ÏÜåÏô∏ Í≥ÑÏ∏µ ÏßÄÏõê Îì± Í≥ÑÌöç ‚ÄúÎ∂ÄÏπú ÏßàÎ≥ë¬∑Í∞ÄÎÇú ÏïÖÏàúÌôò ÎÅäÍ∏∞ ÏõêÌï¥ Íµ≠ÎÇ¥ ÏµúÍ≥† ...
...                                       ...                                                ...
140559  [Í±¥Ï∂ïÍ≥ºÎèÑÏãú] Î∂ÅÌïúÏÇ∞ÏùÑ Ï∫îÎ≤ÑÏä§ ÏÇºÏïÑ. ÎØ∏Ïà†Í¥Ä Îòê ÌïòÎÇòÏùò ÏûëÌíàÏù¥ ÎêòÎã§  <ÏùÄÌèâÍµ¨ ÏßÑÍ¥ÄÎèô ÏÇ¨ÎπÑÎÇò ÎØ∏Ïà†Í¥Ä> ÏÑúÏö∏Ïãú ÏùÄÌèâÍµ¨ ÏßÑÍ¥ÄÎèôÏóê ÏûêÎ¶¨Ïû°ÏùÄ ÏÇ¨ÎπÑÎÇòÎØ∏Ïà†Í¥Ä. ÏÇºÍ∞Å...
140560            Ï°∞ÏÑ†ÌõÑÍ∏∞ Î¨∏Ïù∏ ÍπÄÏ°∞Ïàú Î≥ÑÏû• Í∑∏Î¶∞ Ïò•Ìò∏Ï†ïÎèÑ Ï≤´ Í≥µÍ∞ú  Íµ≠Î¶ΩÏ§ëÏïôÎ∞ïÎ¨ºÍ¥Ä ÏÑúÌôîÏã§ Í∞úÌé∏Ìï¥ 32Ï†ê ÏÉàÎ°≠Í≤å Ï†ÑÏãú Ïò•Ìò∏Ï†ïÎèÑ[Íµ≠Î¶ΩÏ§ëÏïôÎ∞ïÎ¨ºÍ¥Ä Ï†úÍ≥µÏó∞Ìï©Îâ¥Ïä§...
140561                  ÏïàÏÑ± Ï≤≠Î£°ÏÇ¨ ÎåÄÏõÖÏ†ÑÏóêÏÑú Î™©Ïû¨ Í≥°Ïûê Î∞úÍ≤¨  Î¨∏ÌôîÏû¨Ï≤≠(Ï≤≠Ïû• Ï†ïÏû¨Ïàô)Ïùò Íµ≠Í≥†Î≥¥Ï°∞ÏôÄ Í∏∞Ïà†ÏßÄÎèÑÎ°ú ÏïàÏÑ±ÏãúÏóêÏÑú ÏãúÌñâÌïòÍ≥† ÏûàÎäî ÏïàÏÑ± Ï≤≠Î£°ÏÇ¨...
140562          156ÎÖÑÏ†Ñ „Ñ±Ïûê Í≥°Ïûê Ï∞æÏïòÎã§ ÏïàÏÑ± Ï≤≠Î£°ÏÇ¨ Í∏∞Îë• Î∞ëÏóêÏÑú  ÏïàÏÑ± Ï≤≠Î£°ÏÇ¨ ÎåÄÏõÖÏ†ÑÏóêÏÑú Î∞úÍ≤¨Îêú Í≥°Ïûê „ÄêÏÑúÏö∏Îâ¥ÏãúÏä§„Äë Ïù¥ÏàòÏßÄ Í∏∞Ïûê ÏïàÏÑ± Ï≤≠Î£°ÏÇ¨ ÎåÄÏõÖÏ†ÑÏóê...
140563         [ÍπÄÏ§ëÍ∏∞Ïùò ÌïÑÎ¶ÑÌÜµ] ÏÉà ÏòÅÌôî ÎèÑÍµ¥ ÎÇòÏù∏Ïä§ Í≤åÏù¥Ìä∏ ÏïôÏÉÅÎ∏î  ÏòÅÌôî ÎèÑÍµ¥ Ïä§Ìã∏Ïª∑ ‚óÜÎèÑÍµ¥ Í∞êÎèÖ: Î∞ïÏ†ïÎ∞∞ Ï∂úÏó∞: Ïù¥Ï†úÌõà Ï°∞Ïö∞ÏßÑ Ïã†ÌòúÏÑ† ÎèÑÍµ¥ÏùÑ ÏÜåÏû¨Î°ú ...

[140564 rows x 2 columns]

In [9]:
df = df.dropna(axis=0)
df['contents'].count()

140536

In [10]:
df['contents'][0]

'Ï†ïÎßåÌò∏ Íµ≠ÎØºÏÜåÌÜµÏàòÏÑùÏù¥ 12Ïùº Ïò§ÌõÑ Ï≤≠ÏôÄÎåÄ Ï∂òÏ∂îÍ¥Ä ÎåÄÎ∏åÎ¶¨ÌïëÎ£∏ÏóêÏÑú Î≥ÄÏ∞ΩÌù† Íµ≠ÌÜ†Î∂Ä Ïû•Í¥Ä ÏÇ¨Ïùò ÌëúÎ™Ö Í¥ÄÎ†® Î∏åÎ¶¨ÌïëÏùÑ ÌïòÍ≥† ÏûàÎã§. Ïó∞Ìï©Îâ¥Ïä§ Î¨∏Ïû¨Ïù∏ ÎåÄÌÜµÎ†πÏù¥ Î≥ÄÏ∞ΩÌù† Íµ≠ÌÜ†ÍµêÌÜµÎ∂Ä Ïû•Í¥ÄÏùò ÏÇ¨Ïùò ÌëúÎ™ÖÏùÑ ÏÇ¨Ïã§ÏÉÅ ÏàòÏö©ÌñàÎã§. Ï†ïÎßåÌò∏ Ï≤≠ÏôÄÎåÄ Íµ≠ÎØºÏÜåÌÜµÏàòÏÑùÏùÄ 12Ïùº Ïò§ÌõÑ Ï≤≠ÏôÄÎåÄ Ï∂òÏ∂îÍ¥Ä ÎåÄÎ∏åÎ¶¨ÌïëÎ£∏ÏóêÏÑú Î∏åÎ¶¨ÌïëÏùÑ ÌÜµÌï¥ Î≥ÄÏ∞ΩÌù† Íµ≠ÌÜ†ÍµêÌÜµÎ∂Ä Ïû•Í¥ÄÏù¥ Ïù¥ÎÇ† ÌïúÍµ≠ÌÜ†ÏßÄÏ£ºÌÉùÍ≥µÏÇ¨(LH) ÏßÅÏõê Îì±Ïùò 3Í∏∞ Ïã†ÎèÑÏãú ÎïÖ Ìà¨Í∏∞ ÏùòÌòπÍ≥º Í¥ÄÎ†®Ìï¥ Î¨∏Ïû¨Ïù∏ ÎåÄÌÜµÎ†πÏóêÍ≤å ÏÇ¨ÏùòÎ•º ÌëúÎ™ÖÌñàÎã§Í≥† Ï†ÑÌñàÎã§. Î¨∏ ÎåÄÌÜµÎ†πÏùÄ Î≥Ä Ïû•Í¥ÄÏùò ÏÇ¨Ïùò ÌëúÎ™ÖÏóê Ï±ÖÏûÑÏßÄÎäî Î™®ÏäµÏùÑ Î≥¥Ïùº ÏàòÎ∞ñÏóê ÏóÜÎã§Í≥† Î∞ùÌòîÎã§. Ïù¥Ïñ¥ Îã§Îßå 2¬∑4 ÎåÄÏ±ÖÏùò Ï∞®ÏßàÏóÜÎäî Ï∂îÏßÑÏù¥ Îß§Ïö∞ Ï§ëÏöîÌïòÎã§Î©∞ Î≥Ä Ïû•Í¥Ä Ï£ºÎèÑÎ°ú Ï∂îÏßÑÌïú Í≥µÍ≥µÏ£ºÎèÑÌòï Í≥µÍ∏âÎåÄÏ±ÖÍ≥º Í¥ÄÎ†®Îêú ÏûÖÎ≤ïÏùò Í∏∞Ï¥àÏûëÏóÖÍπåÏßÄÎäî ÎßàÎ¨¥Î¶¨Ìï¥Ïïº ÌïúÎã§Í≥† ÎßêÌñàÎã§. Ïù¥Îäî Î¨∏ ÎåÄÌÜµÎ†πÏù¥ ÏÇ¨Ïã§ÏÉÅ Î≥Ä Ïû•Í¥ÄÏùò ÏÇ¨ÏùòÎ•º ÏàòÏö©Ìïú Í≤ÉÏúºÎ°ú Ìï¥ÏÑùÎ

In [11]:
import re
import sys

import nltk
nltk.download('punkt')

# Í∞ÑÎã®Ìïú Ï†ÑÏ≤òÎ¶¨
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace('‚Äù','')
    txt = txt.replace('‚Äú','')
    txt = txt.replace('‚Äô','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()
    
# Í≤ÄÏÇ¨...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))

sentences = []
df = df.dropna(axis=0)
cnt = df['contents'].count()
#print('Total row count:',cnt)
i=0
for raw_text in df['contents']:
    i=i+1
    try:
        if i%100 == 0:
            percent = ("{0:.2f}").format(100 * (i / float(cnt)))
            print(f'\r {percent}% {i}/{str(cnt)}', end="", flush=True)

        docs = nltk.sent_tokenize(clean_text(raw_text))
        for txt in docs:
            if txt.find('‚ñ∂') > -1 or txt.find('@') > -1 or txt.find('‚ìí') > -1: 
                pass
            else:
                txt = txt.strip()
                if any(chr.isdigit() for chr in txt) :
                    pass
                else:
                    sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        pass #print("Unexpected error:", sys.exc_info()[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
 3.70% 5200/140536

KeyboardInterrupt: ignored

In [12]:
import json  
import zipfile  

d1 = None  
data = None  
with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/summary/data/·Ñâ·Ö°·Ñâ·Ö•·ÜØ·Ñå·Ö°·Ü∏·Ñå·Öµ_2.train_original.json.zip", "r") as z:
    with z.open('train_original.json') as f:  
        data = f.read()  
        d1 = json.loads(data.decode("utf-8"))

In [ ]:
for i in range(len(d1)):
    for txt in d1[i]['article_original']:
        sentences.append(txt)

In [13]:
d2 = None  
data = None  
with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/summary/data/·Ñâ·Öµ·Ü´·ÑÜ·ÖÆ·Ü´·ÑÄ·Öµ·Ñâ·Ö°_2.train_original.json.zip", "r") as z:
    with z.open('train_original.json') as f:  
        data = f.read()  
        d2 = json.loads(data.decode("utf-8"))

In [ ]:
' '.join(d2[1]['article_original'])

In [ ]:
for i in range(len(d2)):
    for txt in d1[2]['article_original']:
        sentences.append(txt)

In [ ]:
import re
import sys
import io

#ÌÖçÏä§Ìä∏ Ï†ïÏ†ú(Ï†ÑÏ≤òÎ¶¨)
def cleanText(readData):
    #ÌÖçÏä§Ìä∏Ïóê Ìè¨Ìï®ÎêòÏñ¥ ÏûàÎäî ÌäπÏàò Î¨∏Ïûê Ï†úÍ±∞
    text = re.sub('[-=+,#/\?:^$.@*\"‚Äª~&%„Üç!„Äè\\‚Äò|\(\)\[\]\<\>`\'‚Ä¶„Äã‚óÜ‚óá‚óèüéß‚óã‚ñ≤\t‚Äï‚ñ≥‚îÅ‚ñ∑]', '', readData)
    return text.lower()

In [ ]:
c_sentences = []
for sentence in sentences:
    s = cleanText(sentence)
    c = len(s.split())
    if c >= 3 and c < 10 and s.find('Ïû¨Î∞∞Ìè¨') < 0 and s.find('Í∏∞Ïûê') < 0  and s.find('Ïú†Ìà¨Î∏å') < 0 and s.find('www') < 0 and s.find('com') < 0 and s.find('Ï†ëÏÜçÌïòÍ∏∞') < 0 and s.find('http') < 0 and s.find('Îâ¥Ïä§') < 0 and s.find('ÏùºÎ≥¥') < 0 :
        if s.endswith(('Îã§','Ïöî')):
            c_sentences.append(s.strip())

In [ ]:
len(c_sentences)

In [ ]:
c_sentences[:100]

In [ ]:
ko_sentences_dataset = c_sentences

# Î¨∏Î≤ï discriminatorÏùò ÌïôÏäµ

In [14]:
use_pretrained_model = True

if use_pretrained_model:
    #g_discriminator = Grammar_Discriminator(input_dir = '/content/drive/MyDrive/Colab Notebooks/summary/model_save')
    g_discriminator = Grammar_Discriminator(input_dir = '/content/drive/MyDrive/Colab Notebooks/summary/ko_grammar_model3')
else:
    sentences,labels = collect_training_dataset_for_grammar_discriminator(ko_sentences_dataset)
    print(len(sentences))
    g_discriminator = Grammar_Discriminator()
    g_discriminator.set_dataset(sentences,labels)
    g_discriminator.train(epochs=1)
    g_discriminator.save_model(output_dir='/content/drive/MyDrive/Colab Notebooks/summary/ko_grammar_model3')

Loading BERT tokenizer...


In [15]:
if False: ## Ï∂îÍ∞ÄÏ†ÅÏù∏ fine-tuning
    g_discriminator.train(epochs=10)
    g_discriminator.save_model(output_dir='/content/drive/MyDrive/Colab Notebooks/summary/ko_grammar_model3')

In [16]:
txt = ['ÏµúÍ∑º ÎÇ†Ïî®Í∞Ä Ìè¨Í∑ºÌï¥ÏßÄÎ©¥ÏÑú ÏÇ∞ÏùÑ Ï∞æÎäî ÏÇ¨ÎûåÎì§ÎèÑ ÎäòÍ≥† ÏûàÎäîÎç∞Ïöî','ÏÑúÎπÑÏä§Î•º ÏùåÏõê ÌîåÎû´Ìèº Ïä§Ìè¨Ìã∞ÌååÏù¥Í∞Ä Íµ≠ÎÇ¥ Î°†Ïπ≠ÌïúÎã§']
g_discriminator.discriminator.to(device)
g_discriminator.transfer_learning(txt)

(tensor(4.3092, device='cuda:0', grad_fn=<NllLossBackward>),
 tensor([[-4.0743,  4.2727],
         [ 4.2365, -4.3814]], device='cuda:0', grad_fn=<AddmmBackward>))

##4.3 Static similarity discriminator class

In [17]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
import scipy


class Similarity_Discriminator:
    '''
    _instance = None
    _embedder = None
    def __new__(cls,pre_trained_model_name='stsb-roberta-large'):
        if cls._instance is None:
            print('Creating Similarity_Discriminator object')
            cls._instance = super(Similarity_Discriminator, cls).__new__(cls)
            # Put any initialization here.
            cls._embedder = SentenceTransformer(pre_trained_model_name)
        return cls._instance

    '''

    def __init__(self,pre_trained_model_name='xlm-r-large-en-ko-nli-ststb'):
        print('Creating Similarity_Discriminator object')
        # Put any initialization here.
        self._embedder = SentenceTransformer(pre_trained_model_name)  
        #self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)

    def encode(self,texts):
        return self._embedder.encode(texts,show_progress_bar=False)

    def similarity(self, query_text, org_text_emb):
        queries = nltk.sent_tokenize(query_text)
        query_embeddings = self._embedder.encode(queries,show_progress_bar=False)
        #query_embeddings = self._embedder.encode(queries,show_progress_bar=False)
        #print(queries)
        #print(org_text_emb)
        
        if len(query_embeddings) == 0:
            return 0.0

        cos_scores = scipy.spatial.distance.cdist(query_embeddings, org_text_emb, "cosine")
        similarity_score = 1.0 - np.mean(np.min(cos_scores,axis=0))
        '''
        for query, query_embedding in zip(queries, query_embeddings):
            distances = scipy.spatial.distance.cdist([query_embedding], [org_text_emb], "cosine")[0]
            results = zip(range(len(distances)), distances)
            for idx, distance in results:
                scores.append(1-distance)
        '''
        return similarity_score  
 

###4.3.1 ÌïúÍµ≠Ïñ¥ Î¨∏Ïû• Ïú†ÏÇ¨ÎèÑ pre-trained model Ï†ÅÏö©

In [18]:
s_discriminator = Similarity_Discriminator()
#s_discriminator = Similarity_Discriminator()

Creating Similarity_Discriminator object


Downloading:   0%|          | 0.00/795 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/733 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

# 4.4 Document source class

## Îëê Î¨∏Ïû•ÏùÑ Ìï©ÏπòÎäî rule Î≥ÄÌôòÍ∏∞... --> Ïù¥Í±∞... MLÎ°ú ÎÇòÏ§ëÏóê Î∞îÍøîÏïº...

In [19]:
combine_matching_table = {}

combine_matching_table['Ïñ¥Ïöî.'] = 'Í≥†'
combine_matching_table['ÏßÄÏöî.'] = 'Í≥†'
combine_matching_table['ÎãµÎãàÎã§.'] = 'Í≥†'
combine_matching_table['Î≥¥Í±∞Îùº.'] = 'Î¥ê,'
combine_matching_table['Í∞ÑÎã®Îã§.'] = 'Í∞ÄÎãà,'
combine_matching_table['Îèº.'] = 'Îêò,'
combine_matching_table['Ìï¥.'] = 'ÌïòÎ©∞,'
combine_matching_table['Îã§.'] = 'Í≥†'


In [20]:
def combine_sentence(txt):
    for c in combine_matching_table.keys():
        if txt.endswith(c):
            txt = txt.replace(c,combine_matching_table[c])
    return txt

In [21]:

conjunction_table = ['Í∑∏Îü¨Îçò','Í∑∏ÎûòÏÑú','Í∑∏Îü¨ÎÇò','Í∑∏Îü∞Îç∞','Í∑∏Î¶¨Í≥†','Í∑∏Îû¨ÎçîÎãà','Í∑∏Îü¨ÎãàÍπå','ÌïòÏßÄÎßå','Í∑∏ÎûòÏÑú']

## 4.4.2 source class Íµ¨ÌòÑ

In [22]:


class Source:

    def __init__(self,org_text):
        self.org_text = org_text

    def __crean_text(self, txt):
        txt = txt.replace('\n',' ')
        txt = txt.replace('\r',' ')    
        txt = txt.replace('=','')
        txt = txt.replace('\"','')   
        txt = txt.replace('\'','')
        txt = txt.replace(',','')
        txt = txt.replace('..','')
        txt = txt.replace('...','')
        txt = txt.replace(' .','.')
        txt = txt.replace('.','. ')
        txt = txt.replace('  ',' ')
        txt = txt.replace('  ',' ')    
        txt = txt.replace('  ',' ')   
        txt = txt.replace('  ',' ')           
        txt = txt.replace('  ',' ')
        txt = txt.replace('  ',' ')    
        txt = txt.replace('  ',' ')   
        txt = txt.replace('  ',' ')           
        return txt.strip()

    def set_key_rate(self,s_discriminator):
        #self.full_text = self.__crean_text(self.full_text.strip())
        self.org_text = self.__crean_text(self.org_text.strip())
        print('-'*50)
        print(self.org_text)
        print('-'*50)
        self.org_sentences = np.array(nltk.sent_tokenize(self.org_text))
        for i,sents in enumerate(self.org_sentences):
            for cj in conjunction_table: 
                if sents.startswith(cj):
                    self.org_sentences[i] = sents[len(cj):].strip()

        #self.full_sentences = np.array(nltk.sent_tokenize(self.full_text))
        
        #self.org_term_set = (' ' + self.org_text + ' ').split(' ')
        self.org_term_set = (' '.join(self.org_sentences)).strip().split(' ')
        self.org_source_length = len(self.org_term_set)
        self.term_table = {}
        self.seps = []
        #morp_table = {}

        for index, word in zip(range(len(self.org_term_set)),self.org_term_set):
            self.term_table[index] = word
            if word.endswith(('.','?')):
                self.seps.append(index)
                if self.org_source_length - 1 == index:
                    pass
                else:
                    self.term_table[index] = combine_sentence(word)

        #print(self.term_table.values())
        #print('------------------------------------------------------------------')

        self.s_discriminator = s_discriminator
        # ÏõêÎ¨∏Ïùò embedding...
        self.org_text_emb = self.s_discriminator.encode(self.org_sentences)
        #self.full_text_emb = self.s_discriminator.encode(self.full_sentences)
        #top_n = int(len(self.term_table) * comp_rate)
        #print('top_n',top_n)
        #self.story_peaks = [i+1 for i in range(top_n)]

    def get_org_sample(self, num):
        return self.org_sentences[np.random.choice(len(self.org_sentences), num)]

    def get_source_embedded_code(self):
        return self.org_text_emb

    def get_random_text(self,rate=0.7):
        cnt = int(len(self.term_table) * rate)
        a = list(self.term_table.keys())
        b = np.random.choice(a, cnt)
        c = [fruit for fruit in a if fruit not in b]
        txt = []
        for i in c:
            txt.append(self.term_table[i])
        return ' '.join(txt).strip(), hash(tuple(b))

In [23]:
txt = """
Ìô©Í∏àÎßàÏ∞®Îäî Ìò∏Î∞ïÏúºÎ°ú, Ìù∞ÎßêÏùÄ ÏÉùÏ•êÎ°ú, ÎßàÎ∂ÄÎäî ÎèÑÎßàÎ±ÄÏúºÎ°ú Î≥ÄÌïòÍ≤å ÎèºÏñ¥Ïöî. 
Í∑∏Îü¨ÎãàÍπå Î∞òÎìúÏãú Î∞§ Ïó¥ÎëêÏãúÍ∞Ä ÎêòÍ∏∞ Ï†ÑÏóê ÎèåÏïÑÏôÄÏïº Ìï¥Ïöî.
"""
s = Source(txt)
s.set_key_rate(s_discriminator)
s.get_random_text()

--------------------------------------------------
Ìô©Í∏àÎßàÏ∞®Îäî Ìò∏Î∞ïÏúºÎ°ú Ìù∞ÎßêÏùÄ ÏÉùÏ•êÎ°ú ÎßàÎ∂ÄÎäî ÎèÑÎßàÎ±ÄÏúºÎ°ú Î≥ÄÌïòÍ≤å ÎèºÏñ¥Ïöî. Í∑∏Îü¨ÎãàÍπå Î∞òÎìúÏãú Î∞§ Ïó¥ÎëêÏãúÍ∞Ä ÎêòÍ∏∞ Ï†ÑÏóê ÎèåÏïÑÏôÄÏïº Ìï¥Ïöî.
--------------------------------------------------


('ÏÉùÏ•êÎ°ú ÎßàÎ∂ÄÎäî ÎèÑÎßàÎ±ÄÏúºÎ°ú Î∞òÎìúÏãú Î∞§ Ïó¥ÎëêÏãúÍ∞Ä ÎèåÏïÑÏôÄÏïº Ìï¥Ïöî.', 3723549918677659193)

# 4.5 Generator class

In [24]:
from functools import reduce


# custom weights initialization called on netG and netD
'''
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.06)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(0.0, 0.05)
        m.bias.data.fill_(0)
'''
class Generator(nn.Module):
    """
        Simple Generator w/ MLP
    """
    '''
    def __init__(self, input_size=1024):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, input_size*2),
            nn.BatchNorm1d(input_size*2),
            nn.LeakyReLU(0.2),
            nn.Linear(input_size*2, input_size*3),
            nn.BatchNorm1d(input_size*3),
            nn.LeakyReLU(0.2),
            nn.Linear(input_size*3, input_size*3),
            nn.BatchNorm1d(input_size*3),
            nn.LeakyReLU(0.2),            
            nn.Linear(input_size*3, input_size*2),
            nn.BatchNorm1d(input_size*2),
            nn.LeakyReLU(0.2),
            nn.Linear(input_size*2, input_size),
            #nn.BatchNorm1d(term_length*4),
            nn.Tanh() # -1 ~ 1
        )
    '''
    
    def __init__(self, input_size=1024):
        super(Generator, self).__init__()
        l1 = nn.Linear(input_size, input_size*4)
        l1.weight.data.normal_(0.0, 0.03)
        bn = nn.BatchNorm1d(input_size*4)
        bn.weight.data.normal_(0.0, 0.05)
        bn.bias.data.fill_(0)        
        l2 = nn.Linear(input_size*4, input_size)
        l2.weight.data.normal_(0.01, 0.05)
        self.layer = nn.Sequential(
            l1,
            bn,
            nn.LeakyReLU(0.2),
            l2,
            #nn.BatchNorm1d(term_length*4),
            nn.Tanh() # -1 ~ 1
        )

    def forward(self, x, bias):
        #biased_noise = torch.randn(N,_NOISE_DIM)
        # stroy peakÏóê Ìï¥ÎãπÌïòÎäî termÏóêÍ≤å ÌèâÍ∑†Í∞íÏóê Ìï¥ÎãπÌïòÎäî biasÎ•º Ï∂îÍ∞Ä ÌïúÎã§.
                 
        y_ = self.layer(x)
        y = torch.add(y_,bias)
        #y = nn.Sigmoid()(y)

        return y, y_


# multi-discriminatorÏóê ÎåÄÌïú Adaptive discriminant factor Î•º Íµ¨ÌïòÍ∏∞ ÏúÑÌïú ÌïôÏäµ

ref : https://realpython.com/python-ai-neural-network/

colab ÏàòÏãùÏûÖÎ†• : 

https://wikidocs.net/1679

https://en.wikipedia.org/wiki/Help:Displaying_a_formula#Formatting_using_TeX

Original GANÏùò Î™©Ï†ÅÌï®Ïàò
$$ \min_{G}\max_{D} V(D,G) = E_{x\sim p_{data}(x)}[logD(x)] + E_{z\sim p_{z}(z)}[log(1-D(G(z)))] $$

Multi-Discriminator GANÏùÄ discriminatorÍ∞Ä Í∞Å Î™©Ï†ÅÏóê ÏùòÌïòÏó¨ Ïó¨Îü¨Í∞ú (NÍ∞ú) ÏûàÎã§.
MDGANÏùò Î™©Ï†ÅÌï®Ïàò

$$ \min_{G}\max_{D_{i\sim N}} V(D_{i\sim N},G) = \sum_{i=1}^N \{E_{x\sim p_{data}(x)}[logD_i(x)] + E_{z\sim p_{z}(z)}[log(1-D_i(G(z)))]\} $$

Ïó¨Í∏∞ÏÑú

$$ L(D_i,G) =  E_{x\sim p_{data}(x)}[logD_i(x)] + E_{z\sim p_{z}(z)}[log(1-D_i(G(z)))] $$

Ïù¥ÎùºÌïòÍ≥† Îã®ÏàúÌôî ÌïòÎ©¥

$$ \min_{G}\max_{D_{i\sim N}} V(D_{i\sim N},G) = \sum_{i=1}^N L(D_i,G) $$

ÏôÄ Í∞ôÏù¥ ÎêúÎã§.

Î¨∏Ï†úÏ†êÏùÄ GANÏùò ÌäπÏÑ±ÏÉÅ, ÌäπÏ†ï DiscriminatorÍ∞Ä ÌïôÏäµÏóê ÏûàÏñ¥ ÏßÄÎ∞∞Ï†ÅÏúºÎ°ú loss Ìï®ÏàòÏóê ÏòÅÌñ•ÏùÑ ÎØ∏ÏπòÍ≤å ÎêòÏñ¥ Í∞ÅÍ∞ÅÏùò DiscriminatorÍ∞Ä Í≥®Í≥†Î£® ÌïôÏäµÏóê Ï∞∏Ïó¨ÌïòÏßÄ Î™ªÌïòÍ≥† ÏùòÎèÑÌïòÏßÄ ÏïäÏùÄ Í≤∞Í≥ºÎ•º ÎßåÎì§Í≤å ÎêúÎã§. Ïù¥Îü¨Ìïú Î¨∏Ï†úÏ†êÏùÑ Í∑πÎ≥µÌïòÍ∏∞ ÏúÑÌï¥ Îã§ÏùåÏùò ÎëêÍ∞ÄÏßÄ Ï†úÏïàÏùÑ ÌïúÎã§.

1) Î™©Ï†ÅÌï®ÏàòÏóê Í∞Å Loss Ïóê ÎåÄÌïú ÌëúÏ§ÄÌé∏Ï∞® (standard-deviation) Î•º Î∞òÏòÅÌïòÏó¨ Í∞Å DiscriminatorÏóê ÎåÄÌïú LossÍ∞Ä ÏÉÅÌò∏ Ïú†ÏÇ¨Ìïú ÏàòÏ§ÄÏùÑ Ïú†ÏßÄÌïòÎ©¥ ÌïôÏäµÏù¥ ÏßÑÌñâÎêòÎèÑÎ°ù ÌïúÎã§.

$$ \min_{G}\max_{D_{i\sim N}} V(D_{i\sim N},G) = \sum_{i=1}^N L(D_i,G) + STD_{i \sim N}(L(D_i,G))$$

2) 1)Ïùò Ï†úÏïàÏóê Ï∂îÍ∞ÄÌïòÏó¨, Í∞Å discriminatorÏóê ÏùòÌïú lossÎ•º Ï†úÏñ¥ÌïòÍ∏∞ ÏúÑÌï¥, adaptive discriminant factor (ADF) Î•º Ï†ÅÏö©ÌïòÍ≥†, ÌïôÏäµÏùò ÏßÑÌñâ Í≥ºÏ†ïÏóêÏÑú Ïù¥Î•º ÏµúÏ†ÅÌôî ÌïúÎã§. 

$$ \min_{G}\max_{D_{i\sim N}} V(D_{i\sim N},G) = \sum_{i=1}^N f_iL(D_i,G) + STD_{i \sim N}(L(D_i,G))$$

Ïó¨ÏÑúÏÑú 

fi = adaptive discriminant factor for discriminator i 

Ï§ëÏöîÌïú Í≤ÉÏùÄ, ÌïôÏäµÍ≥ºÏ†ïÏóêÏÑú LiÏùÑ ÏûëÍ≤å (ÌïôÏäµÏùò Î∞©Ìñ•)ÌïòÍ∏∞ ÏúÑÌï¥ÏÑúÎäî fiÎäî Ïó≠ÏúºÎ°ú Ïª§Ï†∏Ïïº ÌïúÎã§. Í∑∏ÎûòÏïº, Ï†ÑÏ≤¥ Loss functionÏóêÏÑú ÎπÑÏ§ëÏù¥ Ï¶ùÎåÄÎêòÏñ¥ Îçî Ï†ÅÍ∑πÏ†ÅÏù∏ ÌïôÏäµÏù¥ Ïù¥Î£®Ïñ¥ ÏßÄÍ≤å ÎêúÎã§. Îî∞ÎùºÏÑú, fiÏùò ÏµúÏ†ÅÌôî Î∞©Ìñ•ÏùÄ Í∏∞Ï°¥Ïùò gradient decentÏôÄ Î∞òÎåÄ Î∞©Ìñ•Ïù¥ ÎêòÏñ¥Ïïº ÌïúÎã§.

$$ f_i^{t+1} = f_i^t + \alpha \frac{\partial V}{\partial f_i}$$

##4.6 Summarizer class (GAN training)

# ÌïôÏäµÍ∏∞...

In [25]:
import random
import math
import numpy as np
from scipy.special import expit
from transformers import get_linear_schedule_with_warmup

class SAM_Summarizer:

    def __init__(self,g_discriminator,s_discriminator):
        self.g_discriminator = g_discriminator
        #self.c_discriminator = c_discriminator
        self.s_discriminator = s_discriminator
        self.m = nn.Sigmoid()

    def ready(self,source):
        self.source = source  
        #self.source.analysis_frame_terms(self.s_discriminator)
        self.generator = Generator(input_size=self.source.org_source_length)
        #self.generator.apply(weights_init)
        return self

    def summarize(self,epochs=10,batch_size=1,learning_rate=2e-4, display = False):
        history = self.__train(epochs,batch_size,learning_rate,display)
        if display:
            plt.figure(figsize=(12, 6))
            plt.plot(history['gen_g_loss'],label='grammar loss')
            plt.plot(history['gen_l_loss'],label='compression loss')
            plt.plot(history['gen_s_loss'],label='similarity loss')

            plt.plot(history['total loss'],label='total loss')
            plt.plot(history['losses std'],label='standard deviation of losses')
            
            #if 'dis_loss' in history:
            #    plt.plot(history['dis_loss'],label='discriminator grammar loss')
            plt.legend()
            plt.show()

        return history

    # textÏùò ÏÉùÏÑ± for torch
    def __text_gen2(self, p_txt, gen_length):
        gtext = []
        sorted_noise, i = torch.sort(p_txt, descending=True)
        order, i = torch.sort(i[:gen_length], descending=False)
        #print(len(order))
        #print(gen_length)
        assert len(order) == gen_length
        order = order.cpu().detach().numpy()
        for k in order:
            gtext.append((self.source.term_table[k],k))
        return gtext

    def __text_gen3(self, p_txt):
        gtext = []

        for order,p in enumerate(p_txt):
            if p > 0.0:
                gtext.append(self.source.term_table[order])
        return gtext

    def __text_gen4(self, p_txt):
        gtext = ""
        indexs = []
        for order,p in enumerate(p_txt):
            if p > 0.0:
                gtext += self.source.term_table[order] + ' '
                indexs.append(order)
        return gtext.strip(),indexs


    def __discrete_gradient(self,weights,use_gpu=False, verbose=0):
        fake_gen_out = torch.zeros(weights.shape).to(device)
        #fake_com_out = torch.zeros(weights.shape).to(device)
        fake_sim_out = torch.zeros(weights.shape).to(device)
        fake_len_out = torch.zeros(weights.shape).to(device) 

        #real_text = self.source.get_org_sample(weights.shape[0])
        fake_outs = []
        #real_outs = []
        apply_order = []
        for i, noise in enumerate(weights):
            #gtext = self.__text_gen2(noise,gen_length)
            gtext,tk = self.__text_gen4(noise)
            fake_outs.append(gtext)
            apply_order.append((i,tk))
  
        D_z_loss, fake_gmr_out=self.g_discriminator.transfer_learning(fake_outs,train_for = False)

        o_sim_out = []
        o_len_out = []
        for fake_text in fake_outs:
            o_sim_out.append(self.s_discriminator.similarity(fake_text,self.source.org_text_emb))

            l = ((1 - len(fake_text.split(' '))/self.source.org_source_length)-0.5) * 2

            o_len_out.append(l)
            #print(1 - len(fake_text.split(' '))/self.source.org_source_length)
            #o_len_out.append(-len(fake_text.split(' '))/self.source.org_source_length)
        
        
        for j, (i,tk) in enumerate(apply_order):

            try:
                '''
                a = torch.tanh( fake_gmr_out[j,1])
                if a > 0 :
                    fake_gen_out[:] = -0.5
                    fake_gen_out[i,tk] = a
                else:
                    fake_gen_out[:] = 0.5
                    fake_gen_out[i,tk] = a
                '''
                fake_gen_out[i,tk] = torch.tanh( fake_gmr_out[j,1])
                fake_sim_out[i,tk] = o_sim_out[j]
                fake_len_out[:] = o_len_out[j]
                fake_len_out[i,tk] = 0 #torch.tensor(fake_text_len/self.source.org_source_length).to(device)
                #fake_len_out[:] = o_len_out[j] #torch.tensor(fake_text_len/self.source.org_source_length).to(device)
                #print(o_len_out[j])
            except Exception as ex:
                print(j,i,tk)
                print(fake_gmr_out)
                raise ex

        return fake_gen_out, fake_sim_out, fake_len_out #fake_com_out, fake_sim_out #, D_z_loss, D_x_loss


    def __train(self, epochs=10,batch_size=10,learning_rate=2e-4,display = False):
        # In the Deepmind paper they use RMSProp however then Adam optimizer
        # improves training time
        #generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        # This method returns a helper function to compute cross entropy loss
        #cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

        # Set the seed value all over the place to make this reproducible.
        seed_val = int(random.random()*100)

        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        
        criterion = nn.BCELoss()
        #D_opt = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
        G_opt = AdamW(self.generator.parameters(),
                        lr = 2e-3, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                        eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                        )
        # Create the learning rate scheduler.
        scheduler = get_linear_schedule_with_warmup(G_opt, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = epochs)
        
        #gen_length = len(self.source.story_peaks) + int(len(self.source.story_peaks)*self.frame_expansion_ratio)
        pb = ProgressBar(epochs,prefix='Train...')
        gen_gmr_loss_history = []
        gen_len_loss_history = []
        gen_sim_loss_history = []
        dis_loss_history = []    
        total_loss_history = []
        losses_std_history = []

        #model Îì§ÏùÄ cudaÎ°ú Î≥¥ÎÇ∏Îã§.
        self.g_discriminator.discriminator.to(device)
        self.g_discriminator.discriminator.eval() # ÌïôÏäµÌïòÏßÄ ÏïäÎäîÎã§...
        #self.c_discriminator.discriminator.to(device)
        #self.c_discriminator.discriminator.eval() # ÌïôÏäµÌïòÏßÄ ÏïäÎäîÎã§...

        self.generator.to(device)       
        self.generator.train()

        #self.bias_w = init_bias
        initial_bias = 0
        G_s_loss = torch.tensor(0)
        #G_c_loss = torch.tensor(0)
        G_g_loss = torch.tensor(0)


        dfs = torch.tensor([ 1.0, 1.0, 1.0], device=device, dtype=torch.float, requires_grad=True)

        for i in range(epochs):
   
            if True:
                noise = torch.randn(batch_size,self.source.org_source_length).to(device) 
                bias = torch.zeros_like(noise).to(device)
                #if i < epochs/4:
                #bias = torch.randn(batch_size,self.source.org_source_length).to(device) / 4 
                #else:
                
                #bias = torch.randn(batch_size,self.source.org_source_length).to(device) 
                sw, sw0 = self.generator(noise,bias)
                with torch.no_grad():                
                    fake_gmr_out, fake_sim_out, fake_len_out = self.__discrete_gradient(sw)

                #print(fake_len_out)

                sw1 = sw * fake_sim_out
                G_s_loss = -torch.mean(sw1) 
                sw2 = sw * fake_gmr_out
                G_g_loss = -torch.mean(sw2) 
                sw3 = sw * fake_len_out
                G_l_loss = -torch.mean(sw3) * 10

                dsc_loss = torch.stack([G_g_loss,G_s_loss,G_l_loss])

                G_loss = torch.dot(dfs,dsc_loss) + torch.std(dsc_loss)
                #G_loss = G_g_loss + G_s_loss

                self.generator.zero_grad()
                G_loss.backward()
                #print('backward:')
                G_opt.step()
                scheduler.step()
                '''
                learning_rate = 0.1
                with torch.no_grad():
                    dfs += learning_rate * dfs.grad
                    dfs.grad = None                    
                    dfs[dfs < 0] = 0.1                
                '''

            gen_gmr_loss_history.append(G_g_loss.cpu().detach().numpy())
            #gen_com_loss_history.append(G_c_loss.cpu().detach().numpy())
            gen_sim_loss_history.append(G_s_loss.cpu().detach().numpy())
            #dis_loss_history.append(D_loss.cpu().detach().numpy())
            gen_len_loss_history.append(G_l_loss.cpu().detach().numpy())

            #pb.printProgress(+1,f'{i+1}/{epochs} epochs, beta:{dfs} Generator / grammar loss:{G_g_loss}  similarity loss:{G_s_loss}') #,   Discriminator grammar_loss:{D_loss}        ')
            #pb.printProgress(+1,'{}/{} epochs, beta:{}, grammar loss:{:.4f}  similarity loss:{:.4f} length loss:{:.4f}'.format(i+1,epochs,dfs,G_g_loss,G_s_loss,G_l_loss)) #,   Discriminator grammar_loss:{D_loss}        ')
            pb.printProgress(+1,'{}/{} epochs,grammar loss:{:.4f}  similarity loss:{:.4f} length loss:{:.4f}'.format(i+1,epochs,G_g_loss,G_s_loss,G_l_loss)) #,   Discriminator grammar_loss:{D_loss}        ')
            
            total_loss_history.append(torch.sum(dsc_loss).item())
            losses_std_history.append(torch.std(dsc_loss).item())

            
        self.generator.eval()
        #self.g_discriminator.discriminator.eval()
        if display:
            plt.figure(figsize=(12, 6))
            xs = np.arange(self.source.org_source_length)
            plt.bar(xs+0.0,sw0[0].cpu().detach().numpy(),label='before activation weights',width=0.2)
            plt.bar(xs+0.2,sw[0].cpu().detach().numpy(),label='after activation weights',width=0.2)
            plt.bar(xs+0.4,bias[0].cpu().detach().numpy(),label='bias weights',width=0.2)         
            plt.legend()        
            plt.show()

        return  {'gen_g_loss':gen_gmr_loss_history,'gen_s_loss':gen_sim_loss_history,'gen_l_loss':gen_len_loss_history,'total loss':total_loss_history,'losses std':losses_std_history} #,'dis_loss':dis_loss_history }

    def get_summary(self, count):
        #texts = []
        self.generator.cpu()
        self.generator.eval()
        #gen_length = len(self.source.story_peaks) + int(len(self.source.story_peaks)*self.frame_expansion_ratio)
        noise = torch.randn(count,self.source.org_source_length)
        bias = torch.zeros_like(noise)

        with torch.no_grad():
            sw,sw0 = self.generator(noise,bias)
            #sw,sw0 = self.generator(noise)

        max_score = 0
        max_sim = 0
        comp_rate = 0
        best_text = ""

        for p_txt in sw:
            gtext = self.__text_gen3(p_txt)
            text = ' '.join(gtext)
            #print('>>',text)
            sim_score = self.s_discriminator.similarity(text,self.source.org_text_emb)
            if sim_score > max_sim:
                best_text = text.strip()
                loss, out=self.g_discriminator.transfer_learning([text],train_for = False)
                max_score = out[0,1].item()
                comp_rate = 1 - len(best_text.split(' '))/self.source.org_source_length
                max_sim = sim_score
            #texts.append([text.strip(),out,sim_score])
        return best_text, max_score, max_sim, comp_rate

    def get_samples(self,count):
        self.generator.cpu()
        self.generator.eval()
        noise = torch.randn(count,self.source.org_source_length)
        bias = torch.zeros_like(noise)
        with torch.no_grad():
            sw,sw0 = self.generator(noise,bias)
        #samples = []
        best_text = ""
        max_score = 0
        for p_txt in sw:
            gtext = self.__text_gen3(p_txt)
            text = ' '.join(gtext).strip()
            #print(text)
            loss, out=self.g_discriminator.transfer_learning([text],train_for = False)
            sim_score = self.s_discriminator.similarity(text,self.source.org_text_emb)    
            comp_rate = 1 - len(text.split(' '))/self.source.org_source_length
            #samples.append((text,out[0,1].item(),sim_score,comp_rate))
            score = out[0,1].item() + sim_score + comp_rate
            if max_score < score:
                max_score = score
                best_text = text
        #return [best_text for i in range(count)], max_score
        return best_text, max_score


#5. Experiment

## ÎπÑÍµê ÎåÄÏÉÅ ÏöîÏïΩ ÏïåÍ≥†Î¶¨Ï¶ò Ï§ÄÎπÑ

In [26]:
def sam_wgan4(text, epochs=50, batch_size=32,display=False, retry = True, retry_count = 0):
    if retry_count > 7:
        raise Exception("Can't summarize the text")

    source = Source(text)
    source.set_key_rate(s_discriminator)
    summarizer = SAM_Summarizer(g_discriminator,s_discriminator)
    summarizer.ready(source)
    hist = summarizer.summarize(epochs,batch_size=2,learning_rate=5e-3,display=display)
    samples, max_score = summarizer.get_samples(batch_size)
    #print(samples)
    if retry and max_score < 4.1:
        print('max score:',max_score)
        return sam_wgan4(text, epochs+10, batch_size,display=display,retry_count=retry_count+1)
    return samples, max_score

In [ ]:
txt = """
ÏòõÎÇ† Ïñ¥Îäê ÏßëÏóê Í∑ÄÏó¨Ïö¥ Ïó¨Ïûê ÏïÑÍ∏∞Í∞Ä ÌÉúÏñ¥ÎÇ¨Ïñ¥Ïöî.
ÏïÑÍ∏∞Îäî Î¨¥Îü≠Î¨¥Îü≠ ÏûêÎùºÏÑú, ÏòàÏÅòÍ≥† ÎßàÏùåÏî® Í≥†Ïö¥ ÏÜåÎÖÄÍ∞Ä ÎêòÏóàÏñ¥Ïöî.
"""
sam_wgan4(txt,epochs=100,display= True,retry = True)

In [ ]:
txt = """
ÏÑúÏÇ∞ÏãúÏùòÌöå(ÏùòÏû• ÏûÑÏû¨Í¥Ä) Í∞ÄÏ∂©Ïàú¬∑Ïù¥ÏàòÏùò ÏùòÏõêÏù¥ (ÏÇ¨)ÌïúÍµ≠ÏßÄÏó≠Ïã†Î¨∏ÌòëÌöåÏóêÏÑú ÏàòÏó¨ÌïòÎäî Ïö∞ÏàòÏùòÏ†ïÎåÄÏÉÅÏùÑ Î∞õÏïòÎã§. Í∞ÄÏ∂©Ïàú ÏùòÏõêÍ≥º Ïù¥ÏàòÏùò ÏùòÏõêÏùÄ 16Ïùº ÌåîÎ¥âÎ©¥ Ìè∞ÌÉÄÎÇò Î¶¨Ï°∞Ìä∏ÏóêÏÑú Ïó¥Î¶∞ ÌïúÍµ≠ÏßÄÏó≠Ïã†Î¨∏ÌòëÌöå ÌïòÍ≥Ñ ÏõåÌÅ¨ÏÉµÏóêÏÑú ÏßÄÏó≠ÏÇ¨Ìöå Î∞úÏ†ÑÏùÑ ÏúÑÌï¥ ÌôúÎ∞úÌïú ÏùòÏ†ïÌôúÎèôÏùÑ ÌéºÏπú Í≥µÎ°úÎ•º Ïù∏Ï†ïÎ∞õÏïÑ Ïö∞ÏàòÏùòÏ†ïÎåÄÏÉÅÏùÑ ÏàòÏÉÅÌñàÎã§.
"""
sam_wgan4(txt,epochs=50,display= False,retry = True)

In [ ]:
full_text = """
ÏòõÎÇ† Ïñ¥Îäê ÏßëÏóê Í∑ÄÏó¨Ïö¥ Ïó¨Ïûê ÏïÑÍ∏∞Í∞Ä ÌÉúÏñ¥ÎÇ¨Ïñ¥Ïöî.
ÏïÑÍ∏∞Îäî Î¨¥Îü≠Î¨¥Îü≠ ÏûêÎùºÏÑú, ÏòàÏÅòÍ≥† ÎßàÏùåÏî® Í≥†Ïö¥ ÏÜåÎÖÄÍ∞Ä ÎêòÏóàÏñ¥Ïöî.
Í∑∏Îü¨Îçò Ïñ¥ÎäêÎÇ†, ÏÜåÎÖÄÏùò Ïñ¥Î®∏ÎãàÍ∞Ä Î≥ëÏù¥Îì§Ïñ¥ Í∑∏Îßå ÏÑ∏ÏÉÅÏùÑ Îñ†ÎÇòÍ≥† ÎßêÏïòÏñ¥Ïöî.
ÏÜåÎÖÄÏùò ÏïÑÎ≤ÑÏßÄÎäî ÌôÄÎ°ú ÎÇ®ÏùÄ ÏÜåÎÖÄÍ∞Ä Í±±Ï†ïÎêòÏóàÏñ¥Ïöî.
Í∑∏ÎûòÏÑú ÏñºÎßà ÏßÄÎÇòÏÑú ÏÉàÏñ¥Î®∏ÎãàÎ•º ÎßûÏù¥ÌñàÏñ¥Ïöî.
ÏÉàÏñ¥Î®∏ÎãàÎäî ÏÜåÎÖÄÎ≥¥Îã§ ÎÇòÏù¥Í∞Ä ÏúÑÏù∏ ÎëêÎ™ÖÏùò Îî∏ÏùÑ Îç∞Î¶¨Í≥† ÏôîÏñ¥Ïöî.
Í∑∏Îü¨ÎÇò ÏÉàÏñ¥Î®∏ÎãàÏôÄ Ïñ∏ÎãàÎì§ÏùÄ ÏÑ±ÏßàÏù¥ Í≥†ÏïΩÌïú Ïã¨Ïà†ÏüÅÏù¥Îì§Ïù¥ÏóàÏñ¥Ïöî.
ÏÉàÏñ¥Î®∏ÎãàÎäî ÏÜåÎÖÄÍ∞Ä ÏûêÍ∏∞ Îî∏Îì§Î≥¥Îã§ ÏòàÏÅòÍ≥† Ï∞©ÌïúÍ≤å Î™ªÎßàÎïÖÌñàÏñ¥Ïöî.
Í∑∏Îü∞Îç∞ Ïù¥Î≤àÏóêÎäî ÏïÑÎ≤ÑÏßÄÎßàÏ†Ä ÎèåÏïÑÍ∞ÄÏÖ®Ïñ¥Ïöî.
ÏÜåÎÖÄÎäî Ïì∏Í≥†, Îã¶Í≥†, ÌïòÎÖÄÏ≤òÎüº ÌïòÎ£® Ï¢ÖÏùº ÏßëÏïàÏùºÏùÑ ÎèÑÎß°ÏïÑ ÌñàÏñ¥Ïöî.
ÏßëÏïàÏùºÏù¥ ÌûòÎì§Ïñ¥ ÏßÄÏπ†ÎïåÎ©¥ ÎÇúÎ°ØÍ∞ÄÏóê ÏïâÏïÑÏÑú Ïû†Ïãú Ïâ¨Í≥§ ÌñàÏñ¥Ïöî.
Í∑∏Îü¨Îçò Ïñ¥ÎäêÎÇ†, ÏôïÍ∂ÅÏóêÏÑú Î¨¥ÎèÑÌöåÍ∞Ä Ïó¥Î†∏Ïñ¥Ïöî.
Ïã†Îç∞Î†êÎùºÏùò ÏßëÏóêÎèÑ Î¨¥ÎèÑÌöå Ï¥àÎåÄÏû•Ïù¥ ÏôîÏñ¥Ïöî.
ÏÉàÏñ¥Î®∏ÎãàÎäî Ïñ∏ÎãàÎì§ÏùÑ Îç∞Î¶¨Í≥† Î¨¥ÎèÑÌöåÏû•ÏúºÎ°ú Îñ†ÎÇ¨Ïñ¥Ïöî.
Ïã†Îç∞Î†êÎùºÎèÑ Î¨¥ÎèÑÌöåÏóê Í∞ÄÍ≥† Ïã∂ÏóàÏñ¥Ïöî.
ÌòºÏûê ÎÇ®ÏùÄ Ïã†Îç∞Î†êÎùºÎäî ÌõåÏ©çÌõåÏ©ç Ïö∏Í∏∞ ÏãúÏûëÌñàÏñ¥Ïöî.
Í∑∏Îïå Ïñ¥ÎîîÏÑ†Í∞Ä ÎßàÎ≤ïÏÇ¨ Ìï†Î®∏ÎãàÍ∞Ä ÎÇòÌÉÄÎÇ¨Ïñ¥Ïöî.
Ïã†Îç∞Î†êÎùºÍ∞Ä Í≥†Í∞úÎ•º Îì§Ïñ¥Î≥¥Îãà, ÎßàÎ≤ïÏÇ¨ Ìï†Î®∏ÎãàÍ∞Ä ÎπôÍ∑∏Î†à ÏõÉÍ≥† ÏûàÏóàÏñ¥Ïöî.
Ìï†Î®∏ÎãàÎäî ÏÜåÎÖÄÎ•º Î¨¥ÎèÑÌöåÏóê Î≥¥ÎÇ¥Ï§ÑÌÖåÎãà Ìò∏Î∞ï ÌïúÍ∞úÏôÄ ÏÉùÏ•ê ÎëêÎßàÎ¶¨, ÎèÑÎßàÎ±ÄÏùÑ Í∞ÄÏßÄÍ≥† Ïò§Îùº ÌñàÏñ¥Ïöî.
ÎßàÎ≤ïÏÇ¨ Ìï†Î®∏ÎãàÍ∞Ä Ïù¥Í≤ÉÎì§ÏùÑ Î≥¥Î©¥ÏÑú Ï£ºÎ¨∏ÏùÑ Ïô∏Ïõ†Ïñ¥Ïöî.
Í∑∏Î¶¨Í≥† ÏßÄÌå°Ïù¥Î°ú Ìò∏Î∞ïÏùÑ Í±¥ÎìúÎ¶¨Ïûê, Ìò∏Î∞ïÏù¥ ÌôîÎ†§Ìïú Ìô©Í∏àÎßàÏ∞®Î°ú Î≥ÄÌñàÏñ¥Ïöî.
Ïù¥Î≤àÏóêÎäî ÏÉùÏ•êÏôÄ ÎèÑÎßàÎ±ÄÏùÑ Í±¥ÎìúÎ†∏Ïñ¥Ïöî.
Í∑∏Îû¨ÎçîÎãà ÏÉùÏ•êÎäî Ìù∞ÎßêÎ°ú, ÎèÑÎßàÎ±ÄÏùÄ Î©ãÏßÑ ÎßàÎ∂ÄÎ°ú Î≥ÄÌñàÏñ¥Ïöî.
Ïã†Îç∞Î†êÎùºÏùò ÎÇ°ÏùÄ Ïò∑ÏùÄ Íµ¨Ïä¨ Ïû•ÏãùÏù¥ Î∞òÏßùÏù¥Îäî ÏòàÏÅú ÎìúÎ†àÏä§Î°ú Î∞îÎÄåÏóàÏñ¥Ïöî.
Ìï†Î®∏ÎãàÎäî Ïã†Îç∞Î†êÎùºÏóêÍ≤å Î∞òÏßùÎ∞òÏßù ÎπõÎÇòÎäî Ïú†Î¶¨Íµ¨ÎëêÎ•º Ïã†Í≤® Ï£ºÏóàÏñ¥Ïöî.
Í∑∏Î¶¨Í≥† Î∞§ Ïó¥ÎëêÏãúÍ∞Ä ÎêòÎ©¥ Î™®Îì†Í≤å Ï≤òÏùåÎåÄÎ°ú ÎèåÏïÑÍ∞ÑÎã§Í≥† ÏïåÎ†§Ï£ºÏóàÏñ¥Ïöî.
Ìô©Í∏àÎßàÏ∞®Îäî Ìò∏Î∞ïÏúºÎ°ú, Ìù∞ÎßêÏùÄ ÏÉùÏ•êÎ°ú, ÎßàÎ∂ÄÎäî ÎèÑÎßàÎ±ÄÏúºÎ°ú Î≥ÄÌïòÍ≤å ÎèºÏñ¥Ïöî. 
Í∑∏Îü¨ÎãàÍπå Î∞òÎìúÏãú Î∞§ Ïó¥ÎëêÏãúÍ∞Ä ÎêòÍ∏∞ Ï†ÑÏóê ÎèåÏïÑÏôÄÏïº Ìï¥Ïöî.
Ïã†Îç∞Î†êÎùºÎäî Ìô©Í∏àÎßàÏ∞®Î•º ÌÉÄÍ≥† ÏôïÍ∂Å Î¨¥ÎèÑÌöåÏû•ÏúºÎ°ú Í∞ÄÏÑú Î©ãÏßÑ ÏôïÏûêÎãòÏùÑ ÎßåÎÇ¨Ïñ¥Ïöî.
ÏôïÏûêÎãòÎèÑ ÏïÑÎ¶ÑÎã§Ïö¥ Ïã†Îç∞Î†êÎùºÏóêÍ≤å ÎßàÏùåÏùÑ ÎπºÏïòÍ≤ºÏñ¥Ïöî.
ÏôïÏûêÎãòÏùÄ Î¨¥ÎèÑÌöåÏû•Ïóê Î™®Ïù∏ Îã§Î•∏ ÏïÑÍ∞ÄÏî®Îì§ÏùÄ Ï≥êÎã§Î≥¥ÏßÄÎèÑ ÏïäÍ≥† Ïã†Îç∞Î†êÎùºÌïòÍ≥†Îßå Ï∂§ÏùÑ Ï∂îÏóàÏñ¥Ïöî.
Ïã†Îç∞Î†êÎùºÎäî ÏôïÏûêÎãòÍ≥º Ï∂§ÏùÑ Ï∂îÎäêÎùº ÏãúÍ∞Ñ Í∞ÄÎäî Ï§ÑÎèÑ Î™∞ÎûêÏñ¥Ïöî.
Ïñ¥ÎäêÎçß ÏãúÍ∞ÑÏù¥ ÌùòÎü¨ Ïó¥ÎëêÏãúÍ∞Ä ÎêòÏóàÏñ¥Ïöî. 
Î≤ΩÏãúÍ≥ÑÏùò Ïó¥ÎëêÏãúÎ•º ÏïåÎ¶¨Îäî Ï¢ÖÏÜåÎ¶¨Ïóê Ïã†Îç∞Î†êÎùºÎäî ÌôîÎì§Ïßù ÎÜÄÎûêÏñ¥Ïöî.
Ïã†Îç∞Î†êÎùºÍ∞Ä ÌóàÎë•ÏßÄÎë• ÏôïÍ∂ÅÏùÑ Îπ†Ï†∏ÎÇòÍ∞ÄÎäîÎç∞, Ïú†Î¶¨Íµ¨Îëê ÌïúÏßùÏù¥ Î≤óÍ≤®Ï°åÏñ¥Ïöî.
ÌïòÏßÄÎßå Íµ¨ÎëêÎ•º Ï£ºÏö∏ ÏãúÍ∞ÑÏù¥ ÏóÜÏóàÏñ¥Ïöî.
Ïã†Îç∞Î†êÎùºÎ•º Îõ∞Ï´ìÏïÑÏò§Îçò ÏôïÏûêÎãòÏùÄ Ï∏µÍ≥ÑÏóêÏÑú Ïú†Î¶¨Íµ¨Îëê ÌïúÏßùÏùÑ Ï£ºÏõ†Ïñ¥Ïöî.
ÏôïÏûêÎãòÏùÄ Ïú†Î¶¨Íµ¨ÎëêÎ•º Í∞ÄÏßÄÍ≥† ÏûÑÍ∏àÎãòÍªò Í∞ÄÏÑú ÎßêÌñàÏñ¥Ïöî.
Ïù¥ Ïú†Î¶¨Íµ¨ÎëêÏùò Ï£ºÏù∏Í≥º Í≤∞ÌòºÌïòÍ≤†Ïñ¥Ïöî.
Í∑∏ÎûòÏÑú Ïã†ÌïòÎì§ÏùÄ Ïú†Î¶¨Íµ¨ÎëêÏùò Ï£ºÏù∏ÏùÑ Ï∞æÏïÑ Ïò® ÎÇòÎùºÎ•º ÎèåÏïÑÎã§ÎÖîÏñ¥Ïöî.
ÎìúÎîîÏñ¥ Ïã†Îç∞Î†êÎùºÏùò ÏßëÏóêÍπåÏßÄ Ïã†ÌïòÎì§Ïù¥ ÎèÑÏ∞©ÌñàÏñ¥Ïöî.
Ïñ∏ÎãàÎì§ÏùÄ Î∞úÏùÑ Ïò§ÎØÄÎ†§ÎèÑ Î≥¥Í≥†, Íµ¨ÎëêÎ•º ÎäòÎ†§ÎèÑ Î≥¥ÏïòÏßÄÎßå ÌïúÎààÏóê Î≥¥Í∏∞ÏóêÎèÑ Ïú†Î¶¨Íµ¨ÎëêÎäî ÎÑàÎ¨¥ ÏûëÏïòÏñ¥Ïöî.
Í∑∏Îïå Ïã†Îç∞Î†êÎùºÍ∞Ä Ï°∞Ïö©Ìûà Îã§Í∞ÄÏôÄ ÏûêÍ∏∞ÎèÑ ÌïúÎ≤à Ïã†Ïñ¥Î≥¥Í≤å Ìï¥Îã¨ÎùºÍ≥† Î∂ÄÌÉÅÌñàÏñ¥Ïöî.
Ïã†Îç∞Î†êÎùºÎäî Ïã†ÌïòÏóêÍ≤åÏÑú Î∞õÏùÄ Ïú†Î¶¨Íµ¨ÎëêÎ•º Ïã†ÏóàÏñ¥Ïöî.
Ïú†Î¶¨Íµ¨ÎëêÎäî Ïã†Îç∞Î†êÎùºÏùò Î∞úÏóê Íº≠ ÎßûÏïòÏñ¥Ïöî.
Ïã†ÌïòÎì§ÏùÄ Ïã†Îç∞Î†êÎùºÎ•º ÏôïÍ∂ÅÏúºÎ°ú Îç∞Î¶¨Í≥† Í∞îÏñ¥Ïöî.
Í∑∏ Îí§ Ïã†Îç∞Î†êÎùºÎäî ÏôïÏûêÎãòÍ≥º Í≤∞ÌòºÌïòÏó¨ Ïò§ÎûòÏò§Îûò ÌñâÎ≥µÌïòÍ≤å ÏÇ¥ÏïòÎåÄÏöî.
"""

In [ ]:
txt = """
ÏòõÎÇ† Ïñ¥Îäê ÏßëÏóê Í∑ÄÏó¨Ïö¥ Ïó¨Ïûê ÏïÑÍ∏∞Í∞Ä ÌÉúÏñ¥ÎÇ¨Ïñ¥Ïöî.
ÏïÑÍ∏∞Îäî Î¨¥Îü≠Î¨¥Îü≠ ÏûêÎùºÏÑú, ÏòàÏÅòÍ≥† ÎßàÏùåÏî® Í≥†Ïö¥ ÏÜåÎÖÄÍ∞Ä ÎêòÏóàÏñ¥Ïöî.
"""
sam_wgan4(txt.strip(),display= True)


# EncoderDecoderModel

In [ ]:
pre_trained_kobert_model_name='kykim/bert-kor-base'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from transformers import EncoderDecoderModel, BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained(pre_trained_kobert_model_name)

In [ ]:
op = tokenizer('ÏòõÎÇ† Ïñ¥Îäê ÏßëÏóê Í∑ÄÏó¨Ïö¥ Ïó¨Ïûê ÏïÑÍ∏∞Í∞Ä ÌÉúÏñ¥ÎÇ¨Ïñ¥Ïöî.[SEP]ÏïÑÍ∏∞Îäî Î¨¥Îü≠Î¨¥Îü≠ ÏûêÎùºÏÑú ÏòàÏÅòÍ≥† ÎßàÏùåÏî® Í≥†Ïö¥ ÏÜåÎÖÄÍ∞Ä ÎêòÏóàÏñ¥Ïöî.', return_tensors="pt",padding="max_length", truncation=True, max_length=64)
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))

# EncoderDecoder ÌïôÏäµ

In [ ]:
from transformers import EncoderDecoderModel, BertTokenizerFast

try:
    del model
    print('delete model')
except Exception as ex:
    pass

use_pretrained_model = False

if use_pretrained_model:
    model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/GAN_ENDE/model")
    print('Load from pre-trained model...')
else:
    model = EncoderDecoderModel.from_encoder_decoder_pretrained(pre_trained_kobert_model_name, pre_trained_kobert_model_name) # initialize Bert2Bert from pre-trained checkpoints
    print('Initialize with ',pre_trained_kobert_model_name)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

N_EPOCHS = 100

In [ ]:
from transformers import BertTokenizer, AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
        
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)

In [ ]:
def train2(model, iterator, optimizer, criterion, scheduler,clip):
    
    model.train()
    
    epoch_loss = 0
    optimizer.zero_grad()
    for i, batch in enumerate(iterator):

        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)
        decoder_input_ids= torch.tensor(batch["decoder_input_ids"]).to(device)
        decoder_attention_mask= torch.tensor(batch["decoder_attention_mask"]).to(device)
        labels= torch.tensor(batch["labels"]).to(device)

            
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        decoder_input_ids=decoder_input_ids, 
                        decoder_attention_mask=decoder_attention_mask,
                        labels=labels)

        loss = outputs.loss #* b_rewards
        loss.backward()
        ##print('loss',loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        scheduler.step()
        
        epoch_loss += loss.item()
        ##print('epoch_loss',epoch_loss)
    return epoch_loss / len(iterator)

In [34]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def generate_summary(text):
    # cut off at BERT max length 512
    sens = nltk.sent_tokenize(text)
    assert(len(sens) == 2)
    inputs = tokenizer(sens[0].strip()+'[SEP]'+sens[1].strip(), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)
    model.eval()
    outputs = model.generate(input_ids, attention_mask=attention_mask).cpu().detach().numpy()[0]
    o=[]
    for token in outputs:
        if token == tokenizer.pad_token_id:
            break
        o.append(token)
    output_str = tokenizer.batch_decode([o], skip_special_tokens=True)[0]

    return output_str

In [27]:
documents = []
#documents.append(full_text)
for i in range(len(d2)):
    documents.append(' '.join(d2[i]['article_original']))

for i in range(len(d1)):
    documents.append(' '.join(d1[i]['article_original']))

In [28]:
documents[1]

'ÏÑúÏÇ∞ÏãúÏùòÌöå(ÏùòÏû• ÏûÑÏû¨Í¥Ä) Í∞ÄÏ∂©Ïàú¬∑Ïù¥ÏàòÏùò ÏùòÏõêÏù¥ (ÏÇ¨)ÌïúÍµ≠ÏßÄÏó≠Ïã†Î¨∏ÌòëÌöåÏóêÏÑú ÏàòÏó¨ÌïòÎäî Ïö∞ÏàòÏùòÏ†ïÎåÄÏÉÅÏùÑ Î∞õÏïòÎã§. Í∞ÄÏ∂©Ïàú ÏùòÏõêÍ≥º Ïù¥ÏàòÏùò ÏùòÏõêÏùÄ 16Ïùº ÌåîÎ¥âÎ©¥ Ìè∞ÌÉÄÎÇò Î¶¨Ï°∞Ìä∏ÏóêÏÑú Ïó¥Î¶∞ ÌïúÍµ≠ÏßÄÏó≠Ïã†Î¨∏ÌòëÌöå ÌïòÍ≥Ñ ÏõåÌÅ¨ÏÉµÏóêÏÑú ÏßÄÏó≠ÏÇ¨Ìöå Î∞úÏ†ÑÏùÑ ÏúÑÌï¥ ÌôúÎ∞úÌïú ÏùòÏ†ïÌôúÎèôÏùÑ ÌéºÏπú Í≥µÎ°úÎ•º Ïù∏Ï†ïÎ∞õÏïÑ Ïö∞ÏàòÏùòÏ†ïÎåÄÏÉÅÏùÑ ÏàòÏÉÅÌñàÎã§. ÏßÄÎÇúÌï¥ 6Ïõî Ï†ú7Ìöå Ï†ÑÍµ≠ÎèôÏãúÏßÄÎ∞©ÏÑ†Í±∞Î•º ÌÜµÌï¥ Îì±ÏõêÌïú Îëê ÏùòÏõêÏùÄ ÏÇ∞ÏóÖÍ±¥ÏÑ§ÏúÑÏõêÌöåÏóêÏÑú Ïó¥Ï†ïÏ†ÅÏù∏ ÏùòÏ†ïÌôúÎèôÏùÑ ÌéºÏπòÍ≥† ÏûàÎã§. Í∞ÄÏ∂©Ïàú ÏùòÏõêÏùÄ 5Î∂ÑÎ∞úÏñ∏, ÌñâÏ†ïÏÇ¨Î¨¥Í∞êÏÇ¨, ÏãúÏ†ïÏßàÎ¨∏ÏùÑ ÌÜµÌï¥ ÏûêÎèôÏ∞® Ïó∞ÎπÑÌÖåÏä§Ìä∏ Ïó∞Íµ¨ÏãúÏÑ§ Ïú†Ïπò, Ï≤úÏàòÎßå ÏóºÌï¥ÌîºÌï¥ Ïû¨Î∞ú Î∞©ÏßÄ, ÏÑúÏÇ∞Ïãú ÎåÄÌëú ÎÜçÏÇ∞Î¨º Ïú°ÏÑ± Îì± ÏßÄÏó≠Ïùò ÌÅ¨Í≥† ÏûëÏùÄ Î¨∏Ï†úÎ•º Í∞úÏÑ†ÌïòÍ∏∞ ÏúÑÌï¥ ÎÖ∏Î†•ÌïòÍ≥† ÏûàÎã§. Ïù¥ÏàòÏùò ÏùòÏõêÏùÄ ÏßÄÎÇú ÌñâÏ†ïÏÇ¨Î¨¥Í∞êÏÇ¨ÏóêÏÑú ÎåÄÏÇ∞Í≥µÎã® Í∏∞ÏóÖ ÏûÑÏõêÏùÑ Ï∞∏Í≥†Ïù∏ÏúºÎ°ú Ï∂úÏÑùÏãúÏºú ÏßÄÏó≠Ïù∏Ïû¨Ï±ÑÏö© Î∞è Í¥ÄÎÇ¥ÏóÖÏ≤¥¬∑ÏûêÏû¨ ÌôúÏö©Ïù

In [29]:


import json
import pickle


use_pretrained_model = True

prepare_dataset_config = {}
dataset_iterator = []

if use_pretrained_model:
    # Opening JSON file
    with open('/content/drive/MyDrive/GAN_ENDE/model/prepare_dataset_config.json','r') as f:
        # returns JSON object as 
        # a dictionary
        prepare_dataset_config = json.load(f)

    with open("/content/drive/MyDrive/GAN_ENDE/data.bin", "rb") as fp:   # Unpickling
        dataset_iterator = pickle.load(fp)

else:
    prepare_dataset_config['curren_doc_num'] = 0
    prepare_dataset_config['total_doc_num'] = len(documents)
    prepare_dataset_config['preparation_time'] = 0

    with open('/content/drive/MyDrive/GAN_ENDE/model/prepare_dataset_config.json', 'w') as fp:
        json.dump(prepare_dataset_config, fp)

    with open("/content/drive/MyDrive/GAN_ENDE/data.bin", "wb") as fp:   #Pickling
        pickle.dump(dataset_iterator, fp)


In [40]:
prepare_dataset_config

{'curren_doc_num': 49611,
 'preparation_time': 52510.49887466431,
 'sentence_count': 6490,
 'total_doc_num': 313962}

In [39]:
len(dataset_iterator)

6490

## dataset ÎßåÎì§Í∏∞...???

In [37]:


for cnt in range(prepare_dataset_config['curren_doc_num'],len(documents)):
    doc_start_time = time.time()

    doc = documents[cnt]
    org_sentences = np.array(nltk.sent_tokenize(doc.strip()))
    first_summary = org_sentences[0] + ' ' + org_sentences[1]
    for i in range(0,len(org_sentences),2):
        txt = org_sentences[i]
        if i < len(org_sentences)-1:
            txt += org_sentences[i+1]

        try:
            summary, max_score = sam_wgan4(txt,epochs=50,batch_size=32)
            print('max_score:{} compress_rate {:4f} text [{}]'.format(max_score,(len(summary)/len(txt)),summary))
            dataset_iterator.append((txt,summary))
            prepare_dataset_config['curren_doc_num'] = cnt+1
        except Exception as ex:
            print(ex)

    with open("/content/drive/MyDrive/GAN_ENDE/data.bin", "wb") as fp:   #Pickling
        pickle.dump(dataset_iterator, fp)
    doc_end_time = time.time()
    sum_mins, sum_secs = epoch_time(doc_start_time, doc_end_time)
    print('')
    print(f'Document:{cnt+1} Summarizing time: {sum_mins}m {sum_secs}s')
    print('')
    
    prepare_dataset_config['preparation_time'] += doc_end_time - doc_start_time
    prepare_dataset_config['sentence_count'] = len(dataset_iterator)
    with open('/content/drive/MyDrive/GAN_ENDE/model/prepare_dataset_config.json', 'w') as fp:
        json.dump(prepare_dataset_config, fp)
    print(prepare_dataset_config)

Ïä§Ìä∏Î¶¨Î∞ç Ï∂úÎ†• ÎÇ¥Ïö©Ïù¥ Í∏∏Ïñ¥ÏÑú ÎßàÏßÄÎßâ 5000Ï§ÑÏù¥ ÏÇ≠Ï†úÎêòÏóàÏäµÎãàÎã§.
--------------------------------------------------
ÏßëÏ£ºÏù∏Ïù¥ ÏûêÍ∏∞ ÏßëÏóê ÏÇ¨Îäî ÎπÑÏú®Ïù∏ ÏûêÍ∞ÄÏ†êÏú†Ïú®ÏùÄ Ï†ÑÏ≤¥ Í∞ÄÍµ¨Ïùò 57. 7%Î°ú 2017ÎÖÑÏóê Ïù¥Ïñ¥ Ïó≠ÎåÄ ÏµúÍ≥† ÏàòÏ§ÄÏùÑ Ïú†ÏßÄÌñàÎã§. Ïö∞Î¶¨ÎÇòÎùºÏóêÏÑ† Ìïú ÏßëÏóê ÌèâÍ∑† 7. 7ÎÖÑ Í±∞Ï£ºÌïòÎäî Í≤ÉÏúºÎ°ú Ï°∞ÏÇ¨ÎêêÍ≥† Ïù¥ÏÇ¨Î•º ÌïòÎäî Í≤ÉÏùÄ ‚ÄòÏãúÏÑ§Ïù¥ÎÇò ÏÑ§ÎπÑ ÏÉÅÌñ•(41. 1%)‚Äô ‚ÄòÏßÅÏ£ºÍ∑ºÏ†ë(31. 0%)‚Äô Îì± ÏûêÎ∞úÏ†ÅÏù∏ ÏõêÏù∏Ïù¥ ÎßéÏïòÎã§.
--------------------------------------------------
CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
--------------------------------------------------
Ï£ºÍ±∞Î≥µÏßÄÎ°úÎìúÎßµ Îì± Ï£ºÍ±∞ Ï∑®ÏïΩÍ≥ÑÏ∏µÏùÑ ÏßÄÏõêÌïòÎäî Ï†ïÎ∂Ä Ï†ïÏ±ÖÏùÄ Ìö®Í≥ºÎ•º ÎÇ¥Í∏∞ ÏãúÏûëÌñàÎã§. ÏßÄÌïò Ïò•ÌÉëÏóê Í±∞Ï£ºÌïòÎäî Ï≤≠ÎÖÑ Í∞ÄÍµ¨ ÎπÑÏ§ëÏù¥ 3. 1%ÏóêÏÑú 2. 

KeyboardInterrupt: ignored

In [ ]:

encoder_max_length = 64
decoder_max_length = 64
batch_size = 32 #4 # 64, 128

CLIP = 1

best_valid_loss = float('inf')
model.to(device)

for cnt in range(experiment_config['curren_doc_num'],len(documents)):
    doc_start_time = time.time()
    doc = documents[cnt]
    dataset_iterator = []
    org_sentences = np.array(nltk.sent_tokenize(doc.strip()))
    first_summary = org_sentences[0] + ' ' + org_sentences[1]
    for i in range(0,len(org_sentences),2):
        txt = org_sentences[i]
        txt2 = txt
        if i < len(org_sentences)-1:
            txt +=  '[SEP]' + org_sentences[i+1]
            txt2 += org_sentences[i+1]

        batch = {}
        batch['input_ids'] = []
        batch['attention_mask'] = []
        batch['decoder_input_ids'] = []
        batch['decoder_attention_mask'] = []
        batch['labels'] = []
        try:
            samples, max_score = sam_wgan4(txt2,epochs=100,batch_size=batch_size)
            print('  max_score:',max_score)
            #for (text,g,s,l) in samples:
            for text in samples:
                inputs = g_discriminator.tokenizer(txt, padding="max_length", truncation=True, max_length=encoder_max_length)
                outputs = g_discriminator.tokenizer(text, padding="max_length", truncation=True, max_length=decoder_max_length)
                batch["input_ids"].append(inputs.input_ids)
                batch["attention_mask"].append(inputs.attention_mask)
                batch["decoder_input_ids"].append(outputs.input_ids)
                batch["decoder_attention_mask"].append(outputs.attention_mask)
                batch["labels"].append(outputs.input_ids.copy())
            dataset_iterator.append(batch)
        except Exception as ex:
            print(ex)

    # Total number of training steps is [number of batches] x [number of epochs]. 
    # (Note that this is not the same as the number of training samples).
    total_steps = len(dataset_iterator) * N_EPOCHS

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    for epoch in range(N_EPOCHS):
        
        start_time = time.time()
        train_loss = train2(model, dataset_iterator, optimizer, criterion, scheduler,CLIP)
        end_time = time.time()
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        #if valid_loss < best_valid_loss:
        #    best_valid_loss = valid_loss
        #    torch.save(model.state_dict(), 'tut6-model.pt')
        
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
        print('\t---------------------------------------------')
        print('\tOrg text:',first_summary)
        sum_txt = generate_summary(first_summary)
        print('\tSummarized text:',sum_txt)
        print('\tCompression rate:{:4f}'.format(len(sum_txt)/len(first_summary)))
        print('\t---------------------------------------------')
        print('\tTarget text:',generate_summary('ÏòõÎÇ† Ïñ¥Îäê ÏßëÏóê Í∑ÄÏó¨Ïö¥ Ïó¨Ïûê ÏïÑÍ∏∞Í∞Ä ÌÉúÏñ¥ÎÇ¨Ïñ¥Ïöî.[SEP]ÏïÑÍ∏∞Îäî Î¨¥Îü≠Î¨¥Îü≠ ÏûêÎùºÏÑú ÏòàÏÅòÍ≥† ÎßàÏùåÏî® Í≥†Ïö¥ ÏÜåÎÖÄÍ∞Ä ÎêòÏóàÏñ¥Ïöî.'))

    doc_end_time = time.time()
    model.save_pretrained("/content/drive/MyDrive/GAN_ENDE/model")
    experiment_config['curren_doc_num'] = cnt+1
    experiment_config['learing_time'] += doc_end_time - doc_start_time
    with open('/content/drive/MyDrive/GAN_ENDE/model/experiment_config.json', 'w') as fp:
        json.dump(experiment_config, fp)
    print(experiment_config)

In [ ]:
print(generate_summary('ÏÑúÏÇ∞ÏãúÏùòÌöå(ÏùòÏû• ÏûÑÏû¨Í¥Ä) Í∞ÄÏ∂©Ïàú¬∑Ïù¥ÏàòÏùò ÏùòÏõêÏù¥ (ÏÇ¨)ÌïúÍµ≠ÏßÄÏó≠Ïã†Î¨∏ÌòëÌöåÏóêÏÑú ÏàòÏó¨ÌïòÎäî Ïö∞ÏàòÏùòÏ†ïÎåÄÏÉÅÏùÑ Î∞õÏïòÎã§. Í∞ÄÏ∂©Ïàú ÏùòÏõêÍ≥º Ïù¥ÏàòÏùò ÏùòÏõêÏùÄ 16Ïùº ÌåîÎ¥âÎ©¥ Ìè∞ÌÉÄÎÇò Î¶¨Ï°∞Ìä∏ÏóêÏÑú Ïó¥Î¶∞ ÌïúÍµ≠ÏßÄÏó≠Ïã†Î¨∏ÌòëÌöå ÌïòÍ≥Ñ ÏõåÌÅ¨ÏÉµÏóêÏÑú ÏßÄÏó≠ÏÇ¨Ìöå Î∞úÏ†ÑÏùÑ ÏúÑÌï¥ ÌôúÎ∞úÌïú ÏùòÏ†ïÌôúÎèôÏùÑ ÌéºÏπú Í≥µÎ°úÎ•º Ïù∏Ï†ïÎ∞õÏïÑ Ïö∞ÏàòÏùòÏ†ïÎåÄÏÉÅÏùÑ ÏàòÏÉÅÌñàÎã§. '))